# Housing Prices in Recessions

Hypothesis: The mean housing prices in university towns is less affected by recessions that those in non-university towns. 

The three following datasets are used:
1. Median home sale prices from the Zillow research data site
2. A list of university towns in the United States from Wikipedia
3. The United States GDP over time in current dollars

## Data Cleaning

In [38]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

In [33]:
# Load university towns data
towns = pd.read_csv('university_towns.txt', sep = '\t', header = None)

# Clean up names
for i in range(len(towns[0])):
    if '(' in towns[0][i]:
        newname = towns[0][i].split('(')[0][:-1]
        towns[0].replace({towns[0][i]:newname}, inplace = True)
    elif '[' in towns[0][i]:
        newname = towns[0][i].split('[')[0]
        towns[0].replace({towns[0][i]:newname}, inplace = True)
towns.columns = ['RegionName']

# Get indices of states
indices = []
statesalph = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
    "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
    "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
    "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
    "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
    "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
    "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
    "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]
    
for j in range(len(statesalph)):           
    stateidx = [i for i in range(len(towns)) if towns.iloc[i]['RegionName'] == statesalph[j]]
    indices.append(stateidx)

newindices = [indices[i][0] for i in range(len(indices))]
newindices.append(len(towns)) 

# Create State array
multiply = []
for i in range(len(newindices) - 1):
    multiply.append(newindices[i+1]-newindices[i])

statecol = []
for i in range(len(statesalph)):
    statecol.append([statesalph[i]]*multiply[i])

flat_list = [item for sublist in statecol for item in sublist]

# Add State array to dataframe
towns['State'] = flat_list

# Drop states from regions
for i in range(len(towns)):
    if towns.loc[i]['RegionName'] == towns.loc[i]['State']:
        towns.drop([i], inplace = True)

# Reorder columns
towns = towns[['State', 'RegionName']]


In [10]:
# Load GDP data
gdp = pd.read_excel('gdplev.xls', skiprows = 5)

# Select wanted rows and columns
gdp = gdp[['Unnamed: 4', 'GDP in billions of chained 2009 dollars.1']]
n = gdp[gdp['Unnamed: 4'] == '2000q1'].index[0]
gdp = gdp.drop(gdp.index[0:n])
gdp = gdp.reset_index(drop=True)

# Find starts of recession
start = []
for i in range(len(gdp)-4):
    value1 = gdp['GDP in billions of chained 2009 dollars.1'][i]
    value2 = gdp['GDP in billions of chained 2009 dollars.1'][i+1]
    value3 = gdp['GDP in billions of chained 2009 dollars.1'][i+2]
    if (value2 < value1) & (value3 < value2):
        start.append(gdp['Unnamed: 4'][i])
start = start[1]

# Look at data after recession start
idx = gdp.index[gdp['Unnamed: 4'] == start][0]
afterstart = gdp[idx:]
afterstart.reset_index(drop=True, inplace = True)

# Find end of recession
end = []
for i in range(len(afterstart)-2):
    value1 = afterstart['GDP in billions of chained 2009 dollars.1'][i]
    value2 = afterstart['GDP in billions of chained 2009 dollars.1'][i+1]
    value3 = afterstart['GDP in billions of chained 2009 dollars.1'][i+2]
    if (value2 > value1) & (value3 > value2):
        end.append(afterstart['Unnamed: 4'][i+2])

end = end[0]

# Recession quarters
startidx = gdp.index[gdp['Unnamed: 4'] == start][0]
endidx = gdp.index[gdp['Unnamed: 4'] == end][0]
recession = gdp.loc[startidx:endidx]

# Find recession bottom
worst = recession['GDP in billions of chained 2009 dollars.1'].min()
worstidx = gdp.index[gdp['GDP in billions of chained 2009 dollars.1'] == worst][0]
bottom = gdp.loc[worstidx]['Unnamed: 4']


In [35]:
# Load data
homes = pd.read_csv('City_Zhvi_AllHomes.csv')

# Look at years 2000-2015
yearswanted = [year for year in homes.columns if year[0] == '2']
colswanted = ['RegionName', 'State'] + yearswanted
homes = homes[colswanted]

# Convert monthly data to quarters
for i in range(16): #loop through years 2000-2015
    if i < 10:
        yearstr = '200' + str(i)
    else: 
        yearstr = '20' + str(i)
    q1 = [yearstr+'-01', yearstr+'-02', yearstr+'-03'] # divide year into quarters
    q2 = [yearstr+'-04', yearstr+'-05', yearstr+'-06']
    q3 = [yearstr+'-07', yearstr+'-08', yearstr+'-09']
    q4 = [yearstr+'-10', yearstr+'-11', yearstr+'-12']
    quarter = [q1, q2, q3, q4]
    for j in range(len(quarter)): #loop through quarters
        homes[yearstr + 'q' + str(j+1)] = homes[quarter[j]].mean(axis=1) #find mean of quarter
# for 2016
q1 = ['2016-01', '2016-02', '2016-03'] # divide year into quarters
q2 = ['2016-04', '2016-05', '2016-06']
q3 = ['2016-07', '2016-08']
quarter = [q1, q2, q3]
for i in range(len(quarter)): #loop through quarters
    homes['2016q' + str(i+1)] = homes[quarter[i]].mean(axis=1)

# Only keep quarters
homesqtr = homes.drop((year for year in homes.columns if '-' in year), axis = 1)
    
# Multi-index by state and city name
# Dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

homesqtr = homesqtr.replace({'State': states})
homesqtr.set_index(['State', 'RegionName'], inplace=True)
homesqtr.sort_index(inplace = True)

## Analysis

Looking at the 2008 recession, the decline or growth of housing prices between the recession start and recession bottom if found for each town. A t-test is used to compare the university town values to the non-univeristy town values.

Reduced market loss is measured by mean price ratio using the equation:

price ratio = (mean price at quarter before recession)/(mean price at recessionbottom)


In [54]:
 # Create new data showing decline or growth of housing prices between start and bottom
data = homesqtr
data['Bottom-Start'] = data[bottom] - data[start]

# Split university and non-university towns
towns_list = towns.to_records(index = False).tolist()
uni = data.loc[towns_list]
non_uni = data.loc[~data.index.isin(towns_list)]

# Run ttest comparing university to non-university town values
pvalue = ttest_ind(uni['Bottom-Start'], non_uni['Bottom-Start'], nan_policy='omit')[1]

# Price Ratio
uni['Price Ratio'] = uni[start]/uni[bottom]
uni_mean = np.mean(uni['Price Ratio'])

non_uni['Price Ratio'] = non_uni[start]/non_uni[bottom]
non_uni_mean = np.mean(non_uni['Price Ratio'])

print('p-value: {}'.format(pvalue))
print('Mean Price Ratio of University Towns: {}'.format(uni_mean))
print('Mean Price Ratio of Non-University Towns: {}'.format(non_uni_mean))

p-value: 0.004325214853459962
Mean Price Ratio of University Towns: 1.0377181366932766
Mean Price Ratio of Non-University Towns: 1.052375653511628


/Users/owner/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  import sys
/Users/owner/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


With a p-value of 0.004, which is less than 0.1, we reject the null hypothesis. This suggests that housing prices in university towns indeed were less affected by the 2008 recession. University towns have a lower mean price ratio, which further indicates reduced market loss.
